# **Understanding Multilingual BERT**

In [ ]:
!pip install torch==1.1.0

     |████████████████████████████████| 676.9MB 24kB/s 
ERROR: torchvision 0.9.1+cu101 has requirement torch==1.8.1, but you'll have torch 1.1.0 which is incompatible.
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.1.0 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101


In [ ]:
%%capture
!pip install transformers==3.5.1

In [ ]:
from transformers import BertTokenizer, BertModel 

### **Download and load the pre-trained M-BERT model**

In [ ]:
model = BertModel.from_pretrained('bert-base-multilingual-cased')

### **Download and load the pre-trained M-BERT model's tokenizer**

### **Define the input sentence. Let us French sentence as an input:**

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

### **Define the input sentence. Let us French sentence as an input:**

In [ ]:
sentence = "C'est une si belle journée"

### **Tokenize the sentence and get the tokens**

In [ ]:
inputs = tokenizer(sentence, return_tensors="pt")

### **Feed the tokens to the model and get the representation**

In [ ]:
hidden_rp, cls_head = model(**inputs)

# **Getting representation of French sentence with FlauBERT**

In [ ]:
from transformers import FlaubertTokenizer, FlaubertModel

### **Let's download and load the pre-trained flaubert_base_cased model**

In [ ]:
model = FlaubertModel.from_pretrained('flaubert/flaubert_base_cased')

### **Next, download and load the tokenizer which is used for pre-training the flaubert_base_cased model:**

In [ ]:
tokenizer = FlaubertTokenizer.from_pretrained('flaubert/flaubert_base_cased')

### **Now let us define the input sentence for which we want to compute the embedding**

In [ ]:
sentence = "Paris est ma ville préférée"

### **Next, we tokenize the sentence and get the token ids:**

In [ ]:
token_ids = tokenizer.encode(sentence)

### **Convert the tokens ids to torch tensors**

In [ ]:
import torch
token_ids = torch.tensor(token_ids).unsqueeze(0)

In [ ]:
token_ids

tensor([[    0,   299,    27,   198,   221, 12714,     1]])

### **Obtain representation of each token in our sentence using the pre-trained flaubert_base_casemodel**

In [ ]:
representation = model(token_ids)[0]

### **Let us check the shape of representation**

In [ ]:
representation.shape

torch.Size([1, 7, 768])

### **First token is CLS, which is sentence representation**

In [ ]:
cls_rep = representation[:, 0, :]

# **Predicting Mask Words using BETO**

In [1]:
!pip install torch==1.1.0

     |████████████████████████████████| 676.9MB 26kB/s 
ERROR: torchvision 0.9.1+cu101 has requirement torch==1.8.1, but you'll have torch 1.1.0 which is incompatible.
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.1.0 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101


In [2]:
%%capture
!pip install transformers==3.5.1

In [6]:
from transformers import pipeline

### **First, we define our masked word prediction pipeline. We pass the task which we want to perform, the pre-trained model, and tokenizers as arguments. As shown in the below code, we are using the dccuchile/bert-base-spanish-wwm-uncased model which is the pretrained BETO model.**

In [7]:
predict_mask = pipeline(
    
    "fill-mask",
    model="dccuchile/bert-base-spanish-wwm-uncased",
    tokenizer="dccuchile/bert-base-spanish-wwm-uncased",
    
)

Some layers from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['pooler/dense/kernel:0', 'pooler/dense/bias:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


### **Lets take a sentence "todos los caminos llevan a roma" and mask first word in a sentence with masked token to our predict_mask pipeline and get the result:**

In [8]:
result = predict_mask('[MASK] los caminos llevan a Roma')

### **Print Results**

In [10]:
print(result)

[{'sequence': '[CLS] todos los caminos llevan a roma [SEP]', 'score': 0.9719892144203186, 'token': 1399, 'token_str': 'todos'}, {'sequence': '[CLS] todas los caminos llevan a roma [SEP]', 'score': 0.007170998025685549, 'token': 1825, 'token_str': 'todas'}, {'sequence': '[CLS] - los caminos llevan a roma [SEP]', 'score': 0.00535197788849473, 'token': 1139, 'token_str': '-'}, {'sequence': '[CLS] todo los caminos llevan a roma [SEP]', 'score': 0.004154033027589321, 'token': 1300, 'token_str': 'todo'}, {'sequence': '[CLS] y los caminos llevan a roma [SEP]', 'score': 0.003964285831898451, 'token': 1040, 'token_str': 'y'}]


# **Next sentence prediction with BERTje**

In [1]:
!pip install torch==1.1.0

In [1]:
!pip install transformers==4.2.0

     |████████████████████████████████| 1.8MB 5.2MB/s 
  Found existing installation: transformers 4.1.1
    Uninstalling transformers-4.1.1:
      Successfully uninstalled transformers-4.1.1


### **Let us see how to use the pre-trained BERTje model for the next sentence prediction task. First, let's import the necessary modules.**

In [2]:
from transformers import BertForNextSentencePrediction, BertTokenizer
from torch.nn.functional import softmax

### **Download and load the BERTje model:**

In [4]:
model = BertForNextSentencePrediction.from_pretrained("wietsedv/bert-base-dutch-cased", from_tf=True)

All TF 2.0 model weights were used when initializing BertForNextSentencePrediction.

Some weights of BertForNextSentencePrediction were not initialized from the TF 2.0 model and are newly initialized: ['bert.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### **Download and load the BERTje Tokenizer**

In [5]:
tokenizer = BertTokenizer.from_pretrained("wietsedv/bert-base-dutch-cased")

### **Define the input sentence pair**

In [6]:
sentence_A = 'Ik woon in Amsterdam'
sentence_B = 'Een geweldige plek'

### **Get the embeddings of the sentence pairs**

In [7]:
embeddings = tokenizer(sentence_A, sentence_B, return_tensors='pt')

### **Compute the logits**

In [8]:
logits = model(**embeddings)[0]

### **Compute the probability using the SoftMax function**

In [9]:
probs = softmax(logits, dim=1)

### **Print the Probability**

In [10]:
print(probs)

tensor([[0.6603, 0.3397]], grad_fn=<SoftmaxBackward>)
